In [1]:
import os
from datetime import datetime as dt

import matplotlib.pyplot as plt
import pandas as pd
import pytorch_lightning as L
import seaborn as sns
import torch
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
from xgboost import XGBRegressor

import hephaestus.single_row_models as sr

os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Load and preprocess the dataset (assuming you have a CSV file)
df = pd.read_csv("./data/diamonds.csv")
df = df.rename(columns={"price": "target"})
df.head()

,carat,cut,color,clarity,depth,table,target,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [2]:
df.shape

(53940, 10)

In [3]:
pd.DataFrame(df.loc[:, "color"])

,color
0,E
1,E
2,E
3,I
4,J
...,...
53935,D
53936,D
53937,D
53938,H


In [4]:
single_row_config = sr.SingleRowConfig.generate(df, "target")
single_row_config

SingleRowConfig(numeric_token='[NUMERIC_EMBEDDING]', numeric_mask='[NUMERIC_MASK]', numeric_col_tokens=['carat', 'depth', 'table', 'x', 'y', 'z'], categorical_col_tokens=['cut', 'color', 'clarity'], tokens=['[PAD]', '[NUMERIC_MASK]', '[MASK]', '[UNK]', '[NUMERIC_EMBEDDING]', 'carat', 'depth', 'table', 'x', 'y', 'z', 'I1', 'SI2', 'F', 'G', 'VS1', 'D', 'IF', 'I', 'H', 'Very Good', 'VVS1', 'Fair', 'E', 'J', 'Ideal', 'VS2', 'SI1', 'Premium', 'Good', 'VVS2', 'cut', 'color', 'clarity'], token_dict={'[PAD]': 0, '[NUMERIC_MASK]': 1, '[MASK]': 2, '[UNK]': 3, '[NUMERIC_EMBEDDING]': 4, 'carat': 5, 'depth': 6, 'table': 7, 'x': 8, 'y': 9, 'z': 10, 'I1': 11, 'SI2': 12, 'F': 13, 'G': 14, 'VS1': 15, 'D': 16, 'IF': 17, 'I': 18, 'H': 19, 'Very Good': 20, 'VVS1': 21, 'Fair': 22, 'E': 23, 'J': 24, 'Ideal': 25, 'VS2': 26, 'SI1': 27, 'Premium': 28, 'Good': 29, 'VVS2': 30, 'cut': 31, 'color': 32, 'clarity': 33}, token_decoder_dict={0: '[PAD]', 1: '[NUMERIC_MASK]', 2: '[MASK]', 3: '[UNK]', 4: '[NUMERIC_EMBEDD

In [5]:
print(df.columns)
dataset = sr.TabularDS(df, single_row_config)
model = sr.TabularRegressor(single_row_config, d_model=64, n_heads=4)
res = model.forward(dataset[0])
print(df.columns)

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'target', 'x',
       'y', 'z'],
      dtype='object')
Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'target', 'x',
       'y', 'z'],
      dtype='object')


In [6]:
# Split the dataset into training and testing sets
# train_df, test_df = train_test_split(df.copy(), test_size=0.2, random_state=42)
train_df, test_df = train_test_split(df.copy(), test_size=0.2, random_state=42)

train_dataset = sr.TabularDS(train_df, single_row_config)
test_dataset = sr.TabularDS(test_df, single_row_config)
model.forward(train_dataset[0])


train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=20,
    shuffle=True,
    collate_fn=sr.training.tabular_collate_fn,
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=20,
    shuffle=True,
    collate_fn=sr.training.tabular_collate_fn,
)

In [7]:
# model = sr.TabularRegressor(single_row_config, d_model=64, n_heads=4)
out = model.forward(dataset[5])
out.shape

torch.Size([1, 1])

In [8]:
out = model.forward(dataset[0:3])
out.shape

torch.Size([3, 1])

In [9]:
df.loc[3, :]

carat         0.29
cut        Premium
color            I
clarity        VS2
depth         62.4
table         58.0
target         334
x              4.2
y             4.23
z             2.63
Name: 3, dtype: object

In [10]:
len(train_dataloader) * 20 - 43152

8

In [11]:
len(train_df)

43152

In [12]:
len(df)

53940

In [13]:
# Get a batch from the training dataloader
sample_batch = next(iter(train_dataloader))

# Print the batch information
print(f"Sample batch type: {type(sample_batch)}")

Sample batch type: <class 'hephaestus.single_row_models.model_data_classes.InputsTarget'>


In [14]:
(
    sample_batch.inputs.numeric.shape,
    sample_batch.inputs.categorical.shape,
    sample_batch.target.shape,
)  # (torch.Size([20, 6]), torch.Size([20, 3]), torch.Size([20]))

(torch.Size([20, 6]), torch.Size([20, 3]), torch.Size([20, 1]))

In [15]:
sample_batch.target.unsqueeze(-1).shape

torch.Size([20, 1, 1])

In [16]:
train_dataset[0:20].inputs.numeric.shape

torch.Size([20, 6])

In [17]:
x = model.training_step(sample_batch, 1)

y_hat.shape=torch.Size([20, 1]), y.shape=torch.Size([20]), y_hat=tensor([[-0.7224],
        [-0.7166],
        [-0.7674],
        [-0.5791],
        [-0.7364],
        [-0.8030],
        [-0.7878],
        [-0.7357],
        [-0.7957],
        [-0.5834],
        [-0.8361],
        [-0.7527],
        [-0.7450],
        [-0.7577],
        [-0.6195],
        [-0.8393],
        [-0.8241],
        [-0.7139],
        [-0.8273],
        [-0.7331]], grad_fn=<AddmmBackward0>), y=tensor([  907.,  4688.,  9139.,   622., 10396.,   729.,  1656.,  8849.,  2964.,
        16688.,   666.,   945.,  1196.,  2723., 10536.,  9789.,  8455.,  1035.,
         8758.,  2401.])


/Users/kailukowiak/Hephaestus/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/kailukowiak/Hephaestus/.venv/lib/python3.13/site-packages/pytorch_lightning/core/module.py:441: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


In [18]:
x

tensor(47791540., grad_fn=<MseLossBackward0>)

In [ ]:
logger = TensorBoardLogger("runs", name=f"{dt.now()}_tabular_encoder_regressor")
early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
trainer = L.Trainer(max_epochs=6, logger=logger, callbacks=[early_stopping])
trainer.fit(model, train_dataset, test_dataset)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type                    | Params | Mode 
------------------------------------------------------------
0 | model   | TabularEncoderRegressor | 229 K  | train
1 | loss_fn | MSELoss                 | 0      | train
------------------------------------------------------------
229 K     Trainable params
0         Non-trainable params
229 K     Total params
0.916     Total estimated model params size (MB)
39        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/kailukowiak/Hephaestus/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/kailukowiak/Hephaestus/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 6. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Training: |          | 0/? [00:00<?, ?it/s]

y_hat.shape=torch.Size([1, 1]), y.shape=torch.Size([]), y_hat=tensor([[-0.8191]], device='mps:0', grad_fn=<LinearBackward0>), y=tensor(16231., device='mps:0')
y_hat.shape=torch.Size([1, 1]), y.shape=torch.Size([]), y_hat=tensor([[1.2027]], device='mps:0', grad_fn=<LinearBackward0>), y=tensor(4540., device='mps:0')
y_hat.shape=torch.Size([1, 1]), y.shape=torch.Size([]), y_hat=tensor([[2.6923]], device='mps:0', grad_fn=<LinearBackward0>), y=tensor(5729., device='mps:0')
y_hat.shape=torch.Size([1, 1]), y.shape=torch.Size([]), y_hat=tensor([[3.9626]], device='mps:0', grad_fn=<LinearBackward0>), y=tensor(6300., device='mps:0')
y_hat.shape=torch.Size([1, 1]), y.shape=torch.Size([]), y_hat=tensor([[5.0616]], device='mps:0', grad_fn=<LinearBackward0>), y=tensor(12968., device='mps:0')
y_hat.shape=torch.Size([1, 1]), y.shape=torch.Size([]), y_hat=tensor([[6.2724]], device='mps:0', grad_fn=<LinearBackward0>), y=tensor(2167., device='mps:0')
y_hat.shape=torch.Size([1, 1]), y.shape=torch.Size([]),

In [ ]:
1 / 0  # Stop here to avoid running the rest of the code

In [ ]:
print("hlskdafjsdal" + "SDFSDFSD")

In [ ]:
model = sr.TabTransformer(dataset, n_heads=8).to(dataset.device)

batch_size = 3
test_num = dataset.X_train_numeric[0:batch_size, :]
test_num_mask = sr.mask_tensor(test_num, model)
test_cat = dataset.X_test_categorical[0:batch_size, :]
test_cat_mask = sr.mask_tensor(test_cat, model)
with torch.no_grad():
    x = model(
        test_num_mask,
        test_cat_mask,
        task="mlm",
    )
x[0].shape, x[1].shape

In [ ]:
# sr.show_mask_pred(0, model, dataset, probability=0.8)

In [ ]:
# Masked Tabular Modeling
base_model_name = "is_model_global2"

model_time = dt.now()
model_time = model_time.strftime("%Y-%m-%dT%H:%M:%S")
model_name = f"{base_model_name}_{model_time}"

model_save_path = "./checkpoints/mtm_models_small.pt"

In [ ]:
model_list = os.listdir("./checkpoints")
if model_save_path.split("/")[-1] in model_list:
    print("Model already exists")
    model_exists = True
else:
    print("Model does not exist")
    model_exists = False

if model_exists:
    model.load_state_dict(torch.load(model_save_path))
else:
    sr.mtm(model, dataset, model_name, epochs=100, batch_size=1000, lr=0.001)
    torch.save(model.state_dict(), model_save_path)

In [ ]:
# regression_performance = sr.fine_tune_model(
#     model, dataset, model_name="FT100", n_rows=100, epochs=100
# )
# regression_performance

In [ ]:
n_train_rows = [
    # 10,
    100,
    1_000,
    2_000,
    5_000,
    10_000,
    15_000,
    30_000,
    # 40_000,
    dataset.X_train.shape[0],
]

In [ ]:
def train_multiple_sizes(pt_model_path, dataset, n_train_rows, n_epochs=100):
    model = sr.TabTransformer(dataset, n_heads=8).to(dataset.device)
    if pt_model_path is not None:
        model.load_state_dict(torch.load(pt_model_path))

    regression_performance = sr.fine_tune_model(
        model,
        dataset,
        model_name=f"ft_{n_train_rows}",
        n_rows=n_train_rows,
        epochs=n_epochs,
    )

    return regression_performance

In [ ]:
hephaestus_results_no_pre_train = []
pbar = tqdm(n_train_rows)
for i in pbar:
    pbar.set_description(f"n_rows: {i}")
    loss = train_multiple_sizes(None, dataset, i, n_epochs=250)
    hephaestus_results_no_pre_train.append(loss)

In [ ]:
no_pt_df = pd.DataFrame(hephaestus_results_no_pre_train)
no_pt_df["model"] = "Hephaestus No Fine Tune"
no_pt_df

In [ ]:
hephaestus_results = []
pbar = tqdm(n_train_rows)
for i in pbar:
    pbar.set_description(f"n_rows: {i}")
    loss = train_multiple_sizes(model_save_path, dataset, i, n_epochs=250)
    hephaestus_results.append(loss)

In [ ]:
hephaestus_df = pd.DataFrame(hephaestus_results)
hephaestus_df["model"] = "Hephaestus"
hephaestus_df

In [ ]:
hephaestus_df.loc[hephaestus_df.n_rows == 1000, "test_loss"].values

In [ ]:
diamonds_data = pd.read_csv("./data/diamonds.csv")

# Encode categorical features using LabelEncoder
label_encoders = {}
categorical_features = ["cut", "color", "clarity"]
for feature in categorical_features:
    le = LabelEncoder()
    diamonds_data[feature] = le.fit_transform(diamonds_data[feature])
    label_encoders[feature] = le

# Split the dataset into features (X) and target (y)
X = diamonds_data.drop("price", axis=1)
y = diamonds_data["price"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Initialize and train the XGBoost regressor
xgb_regressor = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_regressor.fit(
    X_train[0:batch_size],
    y_train[0:batch_size],
)

# Predict on the test set
y_pred = xgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:,.2f}")

# You can also access feature importance scores
# feature_importances = xgb_regressor.feature_importances_
# print("Feature Importance:")
# for feature, importance in zip(X.columns, feature_importances):
#     print(f"{feature}: {importance:.4f}")

In [ ]:
def xgb_tester(train_set_size):
    xgb_regressor = XGBRegressor(n_estimators=120, learning_rate=0.1, random_state=42)
    xgb_regressor.fit(
        X_train[0:train_set_size],
        y_train[0:train_set_size],
    )

    y_pred = xgb_regressor.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return {"n_rows": train_set_size, "test_loss": mse}


xgb_losses = []
for i in tqdm(n_train_rows):
    mse = xgb_tester(i)
    xgb_losses.append(mse)

In [ ]:
xgb_df = pd.DataFrame(xgb_losses)
xgb_df["model"] = "XGBoost"

xgb_df

In [ ]:
loss_df = pd.concat([hephaestus_df, xgb_df, no_pt_df])  # , no_pt_df
loss_df = loss_df.loc[loss_df["n_rows"] != 10]
loss_df.sample(10)

In [ ]:
# Define the colors for each model
# colors = {"Hephaestus": "blue", "XGBoost": "red"}

# Create a figure and axis object
fig, ax = plt.subplots()

# Loop through each model and plot the test loss as a line
for model, group in loss_df.groupby("model"):
    ax.plot(group["n_rows"], group["test_loss"], label=model)

# Set the axis labels and legend
ax.set_xlabel("Number of Rows")
ax.set_ylabel("Test Loss")
ax.legend()
# set x axis to log scale
ax.set_xscale("log")

# Show the plot
plt.show()

In [ ]:
# Define the colors for each model
# colors = {"Hephaestus": "blue", "XGBoost": "red"}

# Create a figure and axis object
fig, ax = plt.subplots()

# Loop through each model and plot the test loss as a line
for model, group in loss_df.loc[loss_df["model"] != "Hephaestus No Fine Tune"].groupby(
    "model"
):
    ax.plot(group["n_rows"], group["test_loss"], label=model)

# Set the axis labels and legend
ax.set_xlabel("Number of Rows")
ax.set_ylabel("Test Loss")
ax.legend()
# set x axis to log scale
ax.set_xscale("log")

# Show the plot
plt.show()

In [ ]:
# Spread the data to have columns for the loss of each model
# loss_df =
loss_percent_df = loss_df.pivot(
    index="n_rows", columns="model", values="test_loss"
).reset_index()
loss_percent_df["percent_improvement"] = (
    loss_percent_df["XGBoost"] - loss_percent_df["Hephaestus"]
) / loss_percent_df["XGBoost"]

In [ ]:
loss_percent_df

In [ ]:
ax = sns.lineplot(data=loss_percent_df, x="n_rows", y="percent_improvement")
plt.axhline(y=0, color="black", linestyle="--")
ax.set_yticks(loss_percent_df["percent_improvement"].round(2))
ax.set_xticks(loss_percent_df["n_rows"])
# ax.set_xscale("log")
# X lables at 45 degree angle
plt.xticks(rotation=45)
# plt.xlabel